In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1706040571660631477
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11568439735328689930
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 824319699269109736
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 18231157137349249777
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 23093306983
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 14193485300755309247
physical_device_desc: "device: 0, name: TITAN RTX, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:GPU:1"
device_type: "GPU"
memory_

In [3]:
import subprocess
n =str(subprocess.check_output(["nvidia-smi", "-L"]))
n

"b'GPU 0: TITAN RTX (UUID: GPU-869e349f-7149-ecfc-622b-1783bead572d)\\nGPU 1: TITAN RTX (UUID: GPU-1fae47ec-d084-5e6a-ec39-ec0990c49fb5)\\n'"

In [1]:
#!pip install jsonlines
import numpy as np
import tensorflow.compat.v1 as tf
# we are using tf2.0a
tf.disable_v2_behavior()
#import tensorflow as tf
import tensorflow_hub as hub
import jsonlines


g = tf.Graph()
with g.as_default():
    text_input = tf.placeholder(dtype=tf.string, shape=[None])
    embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")
    my_result = embed(text_input)
    init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

with tf.device('/gpu:1'):
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True

    session = tf.Session(graph=g,config=config)
    session.run(init_op)


W0519 12:51:01.110423 140065865652032 deprecation.py:323] From /home/cbios/anaconda3/lib/python3.7/site-packages/tensorflow/python/compat/v2_compat.py:63: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def split_list(a_list):
    half = len(a_list)//2
    return a_list[:half], a_list[half:]

In [3]:
def get_features(chunk_list):
    #a,b = split_list(chunk_list)
    #a_feats = session1.run(my_result, feed_dict={text_input: a})
    feats = session.run(my_result, feed_dict={text_input: chunk_list})
    return feats #np.hstack((a_feats,b_feats))
    



In [4]:
import time
start = time.time()
chunk_list = list()
with jsonlines.open('data/webtext.train.jsonl') as reader:
    for obj in reader:
        for k,v in obj.items():
            if k == 'text':
                if len(chunk_list) == 5000:
                    features = get_features(chunk_list)
                    print(len(features))
                    chunk_list = list()
                chunk_list.append(obj['text'])
features = get_features(chunk_list)
print(len(features))
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
00:04:30.07


In [5]:
file_size = !ls -l data/webtext.train.jsonl
file_size

['-rw-r--r-- 1 cbios cbios 679129270 May 18 07:18 webtext.train.jsonl']

In [7]:
67912927/(30+4*60)
# 255311.7

251529.35925925925

In [8]:
250000/(30+4*60)

925.925925925926

In [ ]:
# 939.84